In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import box

from src.stop_explorer import StopExplorer
from src.grid_partitioning import Grid

### Main code

In [ ]:
# Create a GeoDataFrame for the stops dataset.
path_stops = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.stops.parquet'
stop_explorer = StopExplorer(path_stops)

df_stops = stop_explorer.get_df_stops()
display(df_stops)
df_stops.info()

### Plot some stop segments' statistics

In [ ]:
# Show the frequency of the top-20 pairs (hour_start, hour_stop)
prova = stop_explorer.get_stops_temporal_intervals_freqs().head(20)
display(prova)

### Generate a uniform grid over the bounding box enclosing all the stop segments.

In [ ]:
grid = Grid(grid_cell_length_meters = 100)
grid.compute_grid_for_geodata(df_stops)

### Visualize the uniform grid.

In [ ]:

# mappa = grid.generate_grid_map()
# mappa

### Compute a spatial join between stops and grid's cells.

In [ ]:
grid_cells = grid.get_grid()
df_stops = stop_explorer.get_df_stops().copy()

df_stops = df_stops.sjoin(grid_cells, how='inner', predicate='intersects')
df_stops.rename(columns={'index_right':'cell_id'}, inplace=True)
df_stops['cell_id'] = df_stops['cell_id'].astype(np.uint32)

display(df_stops)
display(df_stops.info())

In [ ]:
stats_cells = df_stops.groupby('cell_id').size().reset_index(name='count_stops').sort_values(by='count_stops', ascending=False).reset_index(drop=True)